<h1>Data Processing</h1>

Ziel dieses Notebooks ist, die gescrapeten Daten so zu verarbeiten, dass Sie später thematisch durchsucht werden können. Ein Embeddingvektor liefert genau diese Funktion. Auf eine Anfrage hin wird der Abstand zwischen dem Embedding der Frage und den Embeddings aller anderen Dokumenten berechnet. Dann werden die Dokumente mit den kleinsten Abständen ausgewählt und dem LLM als Kontext mitgegeben. Mithilfe des Wissens dieser Dokumente soll dass LLM dann in der Lage sein die Frage korrekt zu beantworten.
Für die Embeddings benutzen wir [Google Bert](https://blog.google/products/search/search-language-understanding-bert/)

Beispiel:

User: Welche Dozenten unterrichten das Fach Grundlagen der Informatik?

System wählt besten 5 Dokumente aus 

    <Dokument 1>: ... betreute Prof. Dr. Löhr eine Batchelorarbeit in Grundlagen der Informatik...
    <Dokument 2>: Prof. Dr. Weber tel.: 013882664 email: weber@th.de Raum: HQ: 403, Fächer: Grundlagen der Informatik ...
    <Dokument 3> ...
    <Dokument 4> ...
    <Dokument 5> ...
    

Aus der Nutzeranfrage und den Dokumenten wird eine neue Query erstellt, die dem LLM dann final bereitgetellt wird. Diese sieht in etwa so aus:

    
    <Dokument 1>: ... betreute Prof. Dr. Löhr eine Batchelorarbeit in Grundlagen der     Informatik...
    <Dokument 2>: Prof. Dr. Weber tel.: 013882664 email: weber@th.de Raum: HQ: 403, Fächer: Grundlagen der Informatik ...
    <Dokument 3> ...
    <Dokument 4> ...
    <Dokument 5> ...

    Bitte beantworte folgende Frage unter der Berücksichtigung obiger Dokumente:
    Welche Dozenten unterrichten das Fach Grundlagen der Informatik?


Das LLM wird daraufhin hoffentlich korrekt eine Antwort liefern die ähnlich ist zu:

    A: An der TH Nürnberg Georg Simon Ohm unterichten die Professoren Prof. Dr. Löhr und Prof. Dr. Weber das Fach Grundlagen der Informatik.


In [3]:
import sqlite3
import pandas as pd
from transformers import BertModel, BertTokenizer
import torch
from tqdm import tqdm

Zunächst laden wir die Daten aus der Datenbank. Dabei besitzt jedes Dokument als Metadaten den Titel der Webseite, den filenamen und den Text. Diese speichern wir uns in einen Pandas Dataframe

In [4]:
database = 'html.sqlite'
sql = """
SELECT filename, title, text
FROM html_attrs
"""

con = sqlite3.connect(database)
df = pd.read_sql_query(sql, con)
con.close()

print(df.dtypes)
print(df["text"][3])

        Nuremberg Tech is a university with strong regional roots that understands its role in a globalized living, employment, and research community. Our aim is to offer our students degree programmes and a learning environment that provide access to applied research in today’s international context. Therefore, an international and intercultural orientation already forms a key component of life at the university; the development of this characteristic is anchored in our internationalisation strategy .  The following third-party funded projects support the implementation and continued development of the measures described in the strategy to advance internationality at Nuremberg Tech.                           Internationalisation squared (2022 – 2023)  The Internationalisation squared (INT 2 ) project implements and further develops Nuremberg Tech’s internationalisation strategy. Based on a comprehensive analysis of existing Erasmus university partnerships, three to five European part

Zur überprüfung der Texte können wir nun einmal eine Keywordsuche starten. Dieser Ansatz wird außerdem tiefer im Notebook [spacy_keywordextraction](./spacy_keywordextraction.ipynb) verfolgt.

In [5]:
word = "Gallwitz"

[text for text in df["text"] if word in text][:5]

['       Voraussetzungen / Zielgruppe      Welche Studiengänge gibt es an der Fakultät Informatik?         Welche Studiengänge gibt es an der Fakultät Informatik?  Wirtschaftsinformatik Medieninformatik Informatik abgeschlossenes Bachelorstudium in - Wirtschaftsinformatik - Information Systems and Management - Information Technology in Business Computing - Computer Science in Business Computing abgeschlossenes Bachelorstudium in - Medieninformatik - Informatik - Computer Science - Information Technology abgeschlossenes Bachelorstudium in - Informatik - Computer Science - Information Technology mindestens 210 ECTS-Punkte, Nachqualifikation möglich mindestens 210 ECTS-Punkte, Nachqualifikation möglich mindestens 210 ECTS-Punkte, Nachqualifikation möglich Sechssemestrige Bachelorstudiengänge entsprechen 180 ECTS-Punkten. Daher sind im Masterstudium zusätzliche 30 ECTS-Punkte nachzuholen. Fehlende ECTS-Punkte können durch Nachqualifikation ausgeglichen werden. Zugangsvoraussetzung für den 

Jetzt werden wir für jedes Dokument ein eigenes Word embeddings erstellen. Dazu müssen wir zunächst das BERT Model laden.
Das BERT Model ist ein von Google trainiertes mehrschichtiges neuronales Netz, welches ursprünglich dafür entwickelt wurde, dass ???
es ist trainiert auf 10.000+ Büchern
es gibt Modelle "base" und "large"
uncased heißt ohne klein - Großschreibung

Wir brauchen zur vorbereitung die zusätzlichen Token
[SEP] um das Ende eines Satzes zu markieren
[CLS] am Anfang des Texten
[PAD] zum auffüllen der Token 
Außerdem
TokenIDs
MaskIDs - zum filtern der [PAD]
Segment IDs um verschiedene Sätze zu unterscheiden
Posititional Embeddings


In [8]:
model = BertModel.from_pretrained('bert-base-uncased')

testSentence = "In der Bibliothek gibt es 40 Bücher zu Thema Animes"

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokens = tokenizer.tokenize(testSentence)


In [9]:
html_df["tokens"] = [tokenizer.tokenize(text) for text in html_df["text"]]

In [10]:
# database = 'html.sqlite'

# with sqlite3.connect(database) as con:
#     html_df.to_sql('html_attrs', con, index=False, if_exists='replace')

In [20]:
print(html_df.dtype)
print(html_df["tokens"][2])

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [12]:

print(tokens)

['in', 'der', 'bi', '##bl', '##iot', '##he', '##k', 'gi', '##bt', 'es', '40', 'bu', '##cher', 'zu', 'them', '##a', 'anime', '##s']


In [13]:
#tokens = ['[CLS]'] + tokens + ['[SEP]']

In [14]:
attention_mask = [1 if token != "[PAD]" else 0  for token in tokens]

In [15]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

[1999, 4315, 12170, 16558, 25185, 5369, 2243, 21025, 19279, 9686, 2871, 20934, 7474, 16950, 2068, 2050, 8750, 2015]


In [16]:
token_ids = torch.tensor(token_ids).unsqueeze(0)

attention_mask = torch.tensor(attention_mask).unsqueeze(0)

In [17]:
output = model(token_ids, attention_mask=attention_mask)

In [18]:
print(output[2])

IndexError: tuple index out of range

In [ ]:
output[0].shape

torch.Size([1, 18, 768])

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True)


In [ ]:
def proccessSentence(tokens):
    if len(tokens) == 0:
        # Handle the case when the token list is empty, for example, return a default embedding or raise an exception.
        # For demonstration purposes, we'll return a zero tensor as the default embedding.
        return torch.zeros(768)

    # Ensure the token sequence length is no longer than the maximum sequence length the model can handle (512)
    if len(tokens) > 512:
        tokens = tokens[:512]

    # Padding the token sequence to the maximum sequence length if it's shorter
    if len(tokens) < 512:
        tokens += ['[PAD]'] * (512 - len(tokens))

    segmentsDocument_ids = [1] * len(tokens)
    tokenDocument_idss = tokenizer.convert_tokens_to_ids(tokens)
    tokensDocument_tensor = torch.tensor([tokenDocument_idss], dtype=torch.int64)
    segmentsDocument_tensors = torch.tensor([segmentsDocument_ids], dtype=torch.int64)

    with torch.no_grad():
        outputs = model(tokensDocument_tensor, segmentsDocument_tensors)
        hiddenDocuments_states = outputs[2]

    tokenDocuments_vecs = hiddenDocuments_states[-2][0]
    sentenceDocument_embedding = torch.mean(tokenDocuments_vecs, dim=0)
    #print("Our final sentence embedding vector of shape:", sentenceDocument_embedding)

    return sentenceDocument_embedding

html_df["word_embeddings"] = [proccessSentence(tokens).tolist() for tokens in tqdm(html_df["tokens"])]


Our final sentence embedding vector of shape: tensor([-4.5215e-02,  7.9161e-01,  2.1602e-01, -4.3096e-01,  2.3481e-01,
        -8.0480e-01, -3.4056e-01, -1.5219e-01, -1.1433e-01, -4.5773e-01,
         7.1670e-01, -1.2379e+00, -1.2267e+00,  4.7056e-01,  5.3255e-01,
         1.3756e+00,  9.9024e-01, -3.5074e-01, -6.1360e-01,  1.1567e+00,
         1.3404e+00, -1.0304e-01, -1.8010e-01,  1.3796e+00,  7.7544e-01,
         2.4531e-01,  6.1562e-01, -6.0063e-01, -9.7455e-01, -1.0909e-01,
         5.4500e-01, -6.0632e-01,  1.1604e+00, -5.2821e-01,  6.5900e-01,
        -5.6399e-01, -2.0366e-01, -4.0879e-01,  5.7713e-01,  8.1909e-01,
        -1.0264e-01, -9.8268e-01,  2.6789e-01,  2.0379e-01,  8.8713e-01,
        -7.7998e-01,  2.2124e-01, -1.5512e-01,  5.1073e-01,  1.5088e-01,
        -1.0076e+00, -3.1323e-01,  4.6678e-01,  1.0913e-01,  7.6015e-01,
        -2.0106e-01, -7.6841e-01, -1.0449e+00, -9.0801e-01, -5.4555e-01,
         6.3605e-02, -1.1291e-01, -4.2039e-01, -4.8869e-01, -1.2507e-01,
     

KeyboardInterrupt: 

In [ ]:
# database = 'discord_bot/scrap/html.sqlite'

# with sqlite3.connect(database) as con:
#     html_df.to_sql('html_with_embeddings', con, index=False, if_exists='replace')

In [ ]:
#dokument--> bert anwenden für jeden dokument
# question-bert anwenden
question="was macht Gallwitz?"
document=html_df["text"][1]






tokens = tokenizer.tokenize(question)
tokensDocument = tokenizer.tokenize(document)
segments_ids = [1] * len(tokens)
segmentsDocument_ids = [1] * len(tokensDocument)




token_idss = tokenizer.convert_tokens_to_ids(tokens)
tokenDocument_idss = tokenizer.convert_tokens_to_ids(tokensDocument)


tokens_tensor = torch.tensor([token_idss])
segments_tensors = torch.tensor([segments_ids])

tokensDocument_tensor = torch.tensor([tokenDocument_idss])
segmentsDocument_tensors = torch.tensor([segmentsDocument_ids])

with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2]

with torch.no_grad():
    outputs = model(tokensDocument_tensor, segmentsDocument_tensors)
    hiddenDocuments_states = outputs[2]

# print(token_idss)
# print(tokenDocument_idss)

print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))




Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 6
Number of hidden units: 768


In [ ]:
token_embeddings = torch.stack(hidden_states, dim=0)
token_embeddings = torch.squeeze(token_embeddings, dim=1)
token_embeddings = token_embeddings.permute(1,0,2)

token_vecs_sum = []

# For each token in the sentence...
for token in token_embeddings:
    
    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

Shape is: 6 x 768


In [ ]:
token_vecs = hidden_states[-2][0]

# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)
print ("Our final sentence embedding vector of shape:", sentence_embedding)


tokenDocuments_vecs = hiddenDocuments_states[-2][0]

# Calculate the average of all 22 token vectors.
sentenceDocument_embedding = torch.mean(tokenDocuments_vecs, dim=0)
print ("Our final sentence embedding vector of shape:", sentence_embedding)


NameError: name 'hidden_states' is not defined

In [ ]:
from scipy.spatial.distance import cosine

# Calculate the cosine similarity between the word bank 
# in "bank robber" vs "river bank" (different meanings).
diff_bank = 1 - cosine(sentence_embedding, sentenceDocument_embedding)

print('Vector similarity for *different* meanings:  %.2f' % diff_bank)

Vector similarity for *different* meanings:  0.64
